In [1]:
!nvidia-smi

'nvidia-smi' is not recognized as an internal or external command,
operable program or batch file.


In [2]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q


[notice] A new release of pip available: 22.3 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 18.2 MB/s eta 0:00:00
Found existing installation: transformers 4.29.1
Uninstalling transformers-4.29.1:
  Successfully uninstalled transformers-4.29.1
Found existing installation: accelerate 0.19.0
Uninstalling accelerate-0.19.0:
  Successfully uninstalled accelerate-0.19.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached transformers-4.29.1-py3-none-any.whl (7.1 MB)
  Using cached accelerate-0.19.0-py3-none-any.whl (219 kB)


In [3]:
from transformers import pipeline, set_seed
from datasets import load_dataset, load_from_disk, load_metric
import matplotlib.pyplot as plt, pandas as pd

from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

import nltk  # NLP processing library
from nltk.tokenize import sent_tokenize

from tqdm import tqdm  #progress bar
import torch

nltk.download("punkt") # a pre-trained Punkt tokenizer for English

c:\Users\Admin\Documents\Manu\Jupyter_notebooks\NLP\text_summarization\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

In [4]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cpu'

In [5]:
model_ckpt = "Falconsai/medical_summarization"  # T5 Large for Medical Text Summarization https://huggingface.co/Falconsai/medical_summarization
# model_ckpt = "google/pegasus-cnn_dailymail" # model checkpoint from Hugging Face

tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

model_T5 = AutoModelForSeq2SeqLM.from_pretrained(model_ckpt).to(device)

c:\Users\Admin\Documents\Manu\Jupyter_notebooks\NLP\text_summarization\venv\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Admin\.cache\huggingface\hub\models--Falconsai--medical_summarization. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [8]:
from datasets import load_dataset
##  https://www.kaggle.com/datasets/mathurinache/ms2multidocuments (13.9 GB size. Too large dataset size so not used in this project)
## https://huggingface.co/datasets/owkin/medical_knowledge_from_extracts (only 4.16 MB size)
dataset = load_dataset("owkin/medical_knowledge_from_extracts")
dataset

Generating train split: 100%|██████████| 1383/1383 [00:00<00:00, 4346.83 examples/s]


DatasetDict({
    train: Dataset({
        features: ['Question', 'Answer'],
        num_rows: 1383
    })
})

In [10]:
##### Splitting dataset into train, test and validation

from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split

# Convert the dataset to a pandas DataFrame
df = dataset['train'].to_pandas()

# Split the data into train and temp sets (80% train, 20% temp)
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)

# Further split the temp set into validation and test sets (50% of temp each, i.e., 10% of the original data each)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# Convert DataFrames to Dataset
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

# Combine into a DatasetDict
split_dataset = DatasetDict({
    'train': train_dataset,
    'validation': val_dataset,
    'test': test_dataset
})

print(split_dataset)
dataset = split_dataset

DatasetDict({
    train: Dataset({
        features: ['Question', 'Answer', '__index_level_0__'],
        num_rows: 1106
    })
    validation: Dataset({
        features: ['Question', 'Answer', '__index_level_0__'],
        num_rows: 138
    })
    test: Dataset({
        features: ['Question', 'Answer', '__index_level_0__'],
        num_rows: 139
    })
})


In [11]:
split_lengths = [len(dataset[split])for split in dataset]

print(f"Split lengths: {split_lengths}")
print(f"Features: {dataset['train'].column_names}")
print("\nDialogue:")

print(dataset["test"][1]["Question"])

print("\nSummary:")

print(dataset["test"][1]["Answer"])

Split lengths: [1106, 138, 139]
Features: ['Question', 'Answer', '__index_level_0__']

Dialogue:
Your goal is to extract structured information from the user's input that matches the form described below. When extracting information please make sure it matches the type information exactly. Do not add any attributes that do not appear in the schema shown below.

```TypeScript

 { // Information on the clinical trial characteristics from the abstract
 conditions: string // The condition or disease being treated in the clinical trial
 drug_or_intervention: string // The drug or intervention used in the clinical trial
}
```


Please output the extracted information in JSON format. Do not output anything except for the extracted information. Do not add any clarifying information. Do not add any fields that are not in the schema. If the text contains attributes that do not appear in the schema, please ignore them. All output must be in JSON format and follow the schema specified above. Wrap 

In [12]:
def convert_examples_to_features(example_batch):
    input_encodings = tokenizer(example_batch['Question'] , max_length = 1024, truncation = True )

    with tokenizer.as_target_tokenizer():
        target_encodings = tokenizer(example_batch['Answer'], max_length = 128, truncation = True )

    return {
        'input_ids' : input_encodings['input_ids'],
        'attention_mask': input_encodings['attention_mask'],
        'labels': target_encodings['input_ids']
    }


In [13]:
dataset_pt = dataset.map(convert_examples_to_features, batched = True)

Map:   0%|          | 0/1106 [00:00<?, ? examples/s]

c:\Users\Admin\Documents\Manu\Jupyter_notebooks\NLP\text_summarization\venv\Lib\site-packages\transformers\tokenization_utils_base.py:4016: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Map: 100%|██████████| 139/139 [00:00<00:00, 405.46 examples/s]


In [14]:
dataset_pt["train"]

Dataset({
    features: ['Question', 'Answer', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 1106
})

In [15]:
# Training

from transformers import DataCollatorForSeq2Seq
# Data collator that will dynamically pad the inputs received, as well as the labels.
seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_T5)

In [16]:
from transformers import TrainingArguments, Trainer

trainer_args = TrainingArguments(
    output_dir='T5_output', num_train_epochs=1, learning_rate = 1e-4,
    warmup_steps=50,# Number of steps used for a linear warmup from 0 to learning_rate. Overrides any effect of warmup_ratio.
    per_device_train_batch_size=2, per_device_eval_batch_size=2,
    weight_decay=0.01, #  The weight decay to apply (if not zero) to all layers except all bias and LayerNorm weights in AdamW optimizer.
    logging_steps=10, # Number of update steps between two logs if logging_strategy="steps".
    eval_steps=50, #  Number of update steps between two evaluations if evaluation_strategy="steps". Defaults to the same value as logging_steps if not set.
    save_steps=100, # Number of updates steps before two checkpoint saves if save_strategy="steps".
    gradient_accumulation_steps=4, # Number of updates steps to accumulate the gradients for, before performing a backward/update pass.
    fp16 = True,  # Whether to use fp16 16-bit (mixed) precision training instead of 32-bit training.
    group_by_length = True, # group together samples of roughly the same length in the training dataset (to minimize padding applied and be more efficient)
)
'''
output_dir: Directory where the model checkpoints and logs will be saved.
num_train_epochs: Number of epochs to train the model.
warmup_steps: Number of steps to perform learning rate warmup.
per_device_train_batch_size: Batch size for training per device.
per_device_eval_batch_size: Batch size for evaluation per device.
weight_decay: Weight decay for regularization.
logging_steps: Number of steps between logging updates.
evaluation_strategy: Evaluation strategy during training ('steps' means evaluation will be done every eval_steps steps).
eval_steps: Number of steps between evaluations.
save_steps: Number of steps between saving model checkpoints. 1e6 is a very large number, effectively disabling checkpoint saving.
gradient_accumulation_steps: Number of steps to accumulate gradients before performing a backward pass.
'''

"\noutput_dir: Directory where the model checkpoints and logs will be saved.\nnum_train_epochs: Number of epochs to train the model.\nwarmup_steps: Number of steps to perform learning rate warmup.\nper_device_train_batch_size: Batch size for training per device.\nper_device_eval_batch_size: Batch size for evaluation per device.\nweight_decay: Weight decay for regularization.\nlogging_steps: Number of steps between logging updates.\nevaluation_strategy: Evaluation strategy during training ('steps' means evaluation will be done every eval_steps steps).\neval_steps: Number of steps between evaluations.\nsave_steps: Number of steps between saving model checkpoints. 1e6 is a very large number, effectively disabling checkpoint saving.\ngradient_accumulation_steps: Number of steps to accumulate gradients before performing a backward pass.\n"

In [18]:

trainer = Trainer(model=model_T5, args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_pt["train"], 
                  eval_dataset=dataset_pt["validation"])

In [19]:
trainer.train()

  7%|▋         | 10/138 [10:24<1:49:46, 51.46s/it]

{'loss': 4.3578, 'grad_norm': 5.4084906578063965, 'learning_rate': 2e-05, 'epoch': 0.07}


KeyboardInterrupt: 

In [20]:
# Evaluation

def generate_batch_sized_chunks(list_of_elements, batch_size):
    """split the dataset into smaller batches that we can process simultaneously
    Yield successive batch-sized chunks from list_of_elements."""
    for i in range(0, len(list_of_elements), batch_size):
        yield list_of_elements[i : i + batch_size]



def calculate_metric_on_test_ds(dataset, metric, model, tokenizer, 
                               batch_size=16, device=device, 
                               column_text="article", 
                               column_summary="highlights"):
    article_batches = list(generate_batch_sized_chunks(dataset[column_text], batch_size))
    target_batches = list(generate_batch_sized_chunks(dataset[column_summary], batch_size))

    for article_batch, target_batch in tqdm(
        zip(article_batches, target_batches), total=len(article_batches)):
        
        inputs = tokenizer(article_batch, max_length=1024,  truncation=True, 
                        padding="max_length", return_tensors="pt")
        
        summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                         attention_mask=inputs["attention_mask"].to(device), 
                         length_penalty=0.8, num_beams=8, max_length=128)
        ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''
        
        # Finally, we decode the generated texts, 
        # replace the  token, and add the decoded texts with the references to the metric.
        decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True, 
                                clean_up_tokenization_spaces=True) 
               for s in summaries]      
        
        decoded_summaries = [d.replace("", " ") for d in decoded_summaries]
        
        
        metric.add_batch(predictions=decoded_summaries, references=target_batch)
        
    #  Finally compute and return the ROUGE scores.
    score = metric.compute()
    return score


In [21]:
rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rouge_metric = load_metric('rouge')

C:\Users\Admin\AppData\Local\Temp\ipykernel_6376\2696170338.py:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_metric = load_metric('rouge')
c:\Users\Admin\Documents\Manu\Jupyter_notebooks\NLP\text_summarization\venv\Lib\site-packages\datasets\load.py:759: FutureWarning: The repository for rouge contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/rouge/rouge.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [22]:
score = calculate_metric_on_test_ds(
    dataset['test'][0:10], rouge_metric, trainer.model, tokenizer, batch_size = 2, column_text = 'Question', column_summary= 'Answer'
)

rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )

pd.DataFrame(rouge_dict, index = [f'Falconsai_T5'] )

100%|██████████| 5/5 [06:43<00:00, 80.78s/it]


,rouge1,rouge2,rougeL,rougeLsum
Falconsai_T5,0.002079,0.0,0.002079,0.002079


In [18]:
## Save model
model_T5.save_pretrained(model_ckpt)

In [19]:
## Save tokenizer
tokenizer.save_pretrained("tokenizer")

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/spiece.model',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

In [20]:
#Load

tokenizer = AutoTokenizer.from_pretrained("/content/tokenizer")

In [24]:
#Prediction

gen_kwargs = {"length_penalty": 0.8, "num_beams":8, "max_length": 128}



sample_text = dataset["test"][0]["Question"]

reference = dataset["test"][0]["Answer"]

pipe = pipeline("summarization", model=model_ckpt,tokenizer=tokenizer)

## 
print("Dialogue:")
print(sample_text)


print("\nReference Summary:")
print(reference)


print("\nModel Summary:")
print(pipe(sample_text, **gen_kwargs)[0]["summary_text"])

Token indices sequence length is longer than the specified maximum sequence length for this model (680 > 512). Running this sequence through the model will result in indexing errors


Dialogue:
Your goal is to extract structured information from the user's input that matches the form described below. When extracting information please make sure it matches the type information exactly. Do not add any attributes that do not appear in the schema shown below.

```TypeScript

 { // Information on the clinical trial characteristics from the abstract
 conditions: string // The condition or disease being treated in the clinical trial
 drug_or_intervention: string // The drug or intervention used in the clinical trial
}
```


Please output the extracted information in JSON format. Do not output anything except for the extracted information. Do not add any clarifying information. Do not add any fields that are not in the schema. If the text contains attributes that do not appear in the schema, please ignore them. All output must be in JSON format and follow the schema specified above. Wrap the JSON in <json> tags.



Input:Responses to adalimumab in patients with active psori